In [1]:
import pandas as pd

# Load clean datasets

In [2]:
ah_df = pd.read_csv('../data/clean/all_housing.csv')
ah_df

OWN_OCC             LU_DESC UNIT_NUM  ST_NUM          ST_NAME  ZIPCODE  \
0            Y  THREE-FAM DWELLING      NaN   104.0        PUTNAM ST     2128   
1            Y  THREE-FAM DWELLING      NaN   197.0     LEXINGTON ST     2128   
2            N  THREE-FAM DWELLING      NaN   199.0     LEXINGTON ST     2128   
3            N  THREE-FAM DWELLING      NaN   201.0     LEXINGTON ST     2128   
4            Y    TWO-FAM DWELLING      NaN   203.0     LEXINGTON ST     2128   
...        ...                 ...      ...     ...              ...      ...   
275405       N    STRIP CTR STORES      NaN  2198.0  COMMONWEALTH AV     2135   
275406       N    STRIP CTR STORES      NaN  2199.0  COMMONWEALTH AV     2135   
275407       N    STRIP CTR STORES      NaN  2200.0  COMMONWEALTH AV     2135   
275408       N    STRIP CTR STORES      NaN  2201.0  COMMONWEALTH AV     2135   
275409       N   OTHER EXEMPT BLDG      NaN  2203.0  COMMONWEALTH AV     2135   

               CITY  RES_UNITS                   OWNER  
0       EAST BOSTON          0          PASCUCCI CARLO  
1       EAST BOSTON          0       SEMBRANO RODERICK  
2       EAST BOSTON          0         CHEVARRIA ANA S  
3       EAST BOSTON          0   MADDALENI JAMES E, TS  
4       EAST BOSTON          0       DIGIROLAMO JOHN A  
...             ...        ...                     ...  
275405     BRIGHTON          0  GREALISH MARTIN J TRST  
275406     BRIGHTON          0  GREALISH MARTIN J TRST  
275407     BRIGHTON          0  GREALISH MARTIN J TRST  
275408     BRIGHTON          0  GREALISH MARTIN J TRST  
275409     BRIGHTON          0        COMMWLTH OF MASS  

[275410 rows x 9 columns]

In [3]:
ir_df = pd.read_csv('../data/clean/income_restricted_housing.csv')
ir_df

PROJ_NAME  ZIPCODE SECTION8  RES_UNITS  ST_NUM  \
0     Abbot Street/ Shawmut Ave   2119.0      NaN         16   100.0   
1               Academy Homes I   2119.0      NaN        202  1592.0   
2              Academy Homes II   2119.0        Y        236  2926.0   
3           Adams Court Phase A   2126.0      NaN         50    59.0   
4           Adams Court Phase B   2126.0      NaN         45     NaN   
...                         ...      ...      ...        ...     ...   
1440     xxxxxxxx Blue Hill Ave   2119.0      NaN          4   491.0   
1441   YouthBuild Ruskindale Rd   2136.0      NaN          1  1785.0   
1442              YWCA Berkeley   2116.0      NaN        208    40.0   
1443          YWCA Boston Units   2116.0      NaN        115   140.0   
1444          Zebra Condominium   2118.0      NaN         10  1313.0   

                       ST_NAME      CITY  
0                  Shawmut Ave    Boston  
1                 Columbus Ave   Roxbury  
2                Washington St   Roxbury  
3     Msgr Patrick J Lydon Way    Boston  
4                          NaN       NaN  
...                        ...       ...  
1440          Blue Hill Avenue    Boston  
1441         Columbus Ave #500   Roxbury  
1442               Berkeley St    Boston  
1443              Clarendon St    Boston  
1444             Washington St    Boston  

[1445 rows x 7 columns]

# Step 2. Overlay data between DBs

## Create index to join both dataframes

In [4]:
import re

def create_st_index(row):
    street_name = row["ST_NAME"]
    street_number = row["ST_NUM"]

    if street_name != street_name:
        return None

    street_name = street_name.lower()

    # Remove all non-alphanumeric characters
    street_name = re.sub(r'[^a-z0-9\s]', '', street_name)

    # Remove occurances of all these words from the street name
    remove_words = set(["ave","av", "st", "str", "street", "avenue"])
    street_name = ' '.join([word for word in street_name.split() if word not in remove_words])

    if not street_number or street_number != street_number:
        return street_name
    return f"{int(street_number)} {street_name}"

ah_df["join_idx"] = ah_df.apply(create_st_index, axis=1)
ir_df["join_idx"] = ir_df.apply(create_st_index, axis=1)

# Drop nans
ah_df.dropna(inplace=True, subset=["join_idx"])
ir_df.dropna(inplace=True, subset=["join_idx"])

# Set join_idx as a string
ah_df["join_idx"] = ah_df["join_idx"].astype(str)
ir_df["join_idx"] = ir_df["join_idx"].astype(str)

## Filter for housing whose join_index is on the income restricted dataset

In [5]:
# Tag rows where join_idx is in the other dataframe with new column IS_AFFORDABLE
ah_df["IS_AFFORDABLE"] = ah_df["join_idx"].isin(ir_df["join_idx"])

# Save to csv
ah_df.to_csv("../data/clean/result.csv", index=False)

ah_df

OWN_OCC             LU_DESC UNIT_NUM  ST_NUM          ST_NAME  ZIPCODE  \
0            Y  THREE-FAM DWELLING      NaN   104.0        PUTNAM ST     2128   
1            Y  THREE-FAM DWELLING      NaN   197.0     LEXINGTON ST     2128   
2            N  THREE-FAM DWELLING      NaN   199.0     LEXINGTON ST     2128   
3            N  THREE-FAM DWELLING      NaN   201.0     LEXINGTON ST     2128   
4            Y    TWO-FAM DWELLING      NaN   203.0     LEXINGTON ST     2128   
...        ...                 ...      ...     ...              ...      ...   
275405       N    STRIP CTR STORES      NaN  2198.0  COMMONWEALTH AV     2135   
275406       N    STRIP CTR STORES      NaN  2199.0  COMMONWEALTH AV     2135   
275407       N    STRIP CTR STORES      NaN  2200.0  COMMONWEALTH AV     2135   
275408       N    STRIP CTR STORES      NaN  2201.0  COMMONWEALTH AV     2135   
275409       N   OTHER EXEMPT BLDG      NaN  2203.0  COMMONWEALTH AV     2135   

               CITY  RES_UNITS                   OWNER           join_idx  \
0       EAST BOSTON          0          PASCUCCI CARLO         104 putnam   
1       EAST BOSTON          0       SEMBRANO RODERICK      197 lexington   
2       EAST BOSTON          0         CHEVARRIA ANA S      199 lexington   
3       EAST BOSTON          0   MADDALENI JAMES E, TS      201 lexington   
4       EAST BOSTON          0       DIGIROLAMO JOHN A      203 lexington   
...             ...        ...                     ...                ...   
275405     BRIGHTON          0  GREALISH MARTIN J TRST  2198 commonwealth   
275406     BRIGHTON          0  GREALISH MARTIN J TRST  2199 commonwealth   
275407     BRIGHTON          0  GREALISH MARTIN J TRST  2200 commonwealth   
275408     BRIGHTON          0  GREALISH MARTIN J TRST  2201 commonwealth   
275409     BRIGHTON          0        COMMWLTH OF MASS  2203 commonwealth   

        IS_AFFORDABLE  
0               False  
1               False  
2               False  
3               False  
4               False  
...               ...  
275405          False  
275406          False  
275407          False  
275408          False  
275409          False  

[275410 rows x 11 columns]